# Objectives
* Learn to preprocess the data for Neural Machine Translation system
* Learn to train NMT using encoder-decoder model
* Learn to use the trained model to create inference
* Reference: https://keras.io 
https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

In [ ]:
import numpy as np
import keras, tensorflow
from keras.models import Model
from keras.layers import Input, LSTM, Dense

Using TensorFlow backend.


##**Section 1**

---

## **Data Preparation **

**Task 1**

---


There are few datasets for langauge trasnaltion task are available from Tatoeba Project (http://www.manythings.org/anki/ ).

*  Download preferred langauge pair
*  Extract the file and upload it to colab
*  Create a list of lines by splitting the text file at every occurance of '\n'
*  Print number of sentences
*  Print 100th sentence in original script



In [ ]:
!wget http://www.manythings.org/anki/nld-eng.zip
!unzip nld-eng.zip

--2021-04-15 19:46:04--  http://www.manythings.org/anki/nld-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.55.222, 172.67.173.198, 2606:4700:3031::6815:37de, ...
Connecting to www.manythings.org (www.manythings.org)|104.21.55.222|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1975354 (1.9M) [application/zip]
Saving to: ‘nld-eng.zip.2’

nld-eng.zip.2       100%[===================>]   1.88M  1.04MB/s    in 1.8s    

2021-04-15 19:46:06 (1.04 MB/s) - ‘nld-eng.zip.2’ saved [1975354/1975354]

Archive:  nld-eng.zip
  inflating: _about.txt              
  inflating: nld.txt                 


In [ ]:
filename="nld.txt"
with open(filename , 'r') as tamil_file:
  lines=tamil_file.read().split('\n')

In [ ]:
print(len(lines))
print(lines[100])
#lines

28548
I'm back.	Ik ben weer thuis.


**Task 2**

---

* Split each line into input text and target text
* Add '\t' to denote begning of sentence and '\n'  or '<eos\>' to denote end of the sentence to the each target line.
* Compute every set of character in the input text and target text
* Print input characters and target characters as sorted list

In [ ]:
input_texts = list()
target_texts = list()
input_characters = set()
target_characters = set()


In [ ]:
num_samples = 10000
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
print("Input characters",input_characters)
print("Target characters",target_characters)


('Number of samples:', 10000)
('Number of unique input tokens:', 74)
('Number of unique output tokens:', 81)
('Max sequence length for inputs:', 24)
('Max sequence length for outputs:', 58)
('Input characters', [' ', '!', '"', '$', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x82', '\xa9', '\xac', '\xc3', '\xe2'])
('Target characters', ['\t', '\n', ' ', '!', '"', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x82', '\xa8', '\xa9'

**Task 3**

---

*  Tokenize our characters by assigning each unique character to an integer value.
*  Create one-hot encoding
*  Create numeric data with one-hot encoding

In [ ]:
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index= dict([(char,i) for i, char in enumerate(target_characters)])

In [ ]:
input_token_index

The encoder_input_data will consist of  samples of the maximum sequence length (24) filled with the respective one-hot-encoded tokens (in this case a vector of length 74). The decoder_input_data and the decoder_target_data are both constructed in the same way as the input data for the encoder. We need to construct those two sequences because we're training our model through a process called teacher forcing, where the decoder learns to generate decoder_target_data[t+1...] given decoder_input_data[...t] while taking into account the input sequence via the encoder's internal state. Therefore we have to offset decoder_target_data by one timestep.

In [ ]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [ ]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 24, 74)
(10000, 58, 81)
(10000, 58, 81)


In [ ]:
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]]=1
    
  for t, char in enumerate(target_text):
    decoder_input_data[i,t,target_token_index[char]]=1
    if t>0:
      decoder_target_data[i,t-1,target_token_index[char]]=1

In [ ]:
print(encoder_input_data[0][0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


## **Section 2**


---

## **Building the Model **

**Hyperparameters **

** Use GPU/TPU to run the model **

In [ ]:
batch_size=64
epochs= 2
latent_dim =256 # latent dimensionality of the encoding space

In [ ]:
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder_lstm= LSTM(latent_dim,return_state=True) #
encoder_outputs,state_h,state_c= encoder_lstm(encoder_inputs)
encoder_states= [state_h,state_c]

In [ ]:
decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_output,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)

decoder_dense= Dense(num_decoder_tokens, activation='softmax')
decoder_outputs=decoder_dense(decoder_output)

In [ ]:
model= Model(inputs=[encoder_inputs,decoder_inputs],outputs=decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
model.summary()

## ** Section 3 **


---

** Training Model **

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save('seq2seq_eng-nl.h5')

# Testing Machine Translation


---

* Use the compiled model

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.load_weights('seq2seq_eng-nl.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
  decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs] + decoder_states)

In [ ]:
# reverse-lookup token index to turn sequences back to characters
reverse_input_char_index = dict(
  (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
  (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
  # encode the input sequence to get the internal state vectors.
  states_value = encoder_model.predict(input_seq)
  
  # generate empty target sequence of length 1 with only the start character
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, target_token_index['\t']] = 1.
  
  # output sequence loop
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
      [target_seq] + states_value)
    
    # sample a token and add the corresponding character to the 
    # decoded sequence
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char
    
    # check for the exit condition: either hitting max length
    # or predicting the 'stop' character
    if (sampled_char == '\n' or 
        len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True
      
    # update the target sequence (length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.
    
    # update states
    states_value = [h, c]
    
  return decoded_sentence

In [ ]:
for seq_index in range(10):
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence)

-
('Input sentence:', 'Hi.')
('Decoded sentence:', 'Hoi.\n')
-
('Input sentence:', 'Run!')
('Decoded sentence:', 'Gewrader ok niet in huister?\n')
-
('Input sentence:', 'Who?')
('Decoded sentence:', 'Wie?\n')
-
('Input sentence:', 'Wow!')
('Decoded sentence:', "Wa's niet gek!\n")
-
('Input sentence:', 'Fire!')
('Decoded sentence:', 'Vuur!\n')
-
('Input sentence:', 'Fire!')
('Decoded sentence:', 'Vuur!\n')
-
('Input sentence:', 'Help!')
('Decoded sentence:', 'Help!\n')
-
('Input sentence:', 'Jump.')
('Decoded sentence:', 'Spring.\n')
-
('Input sentence:', 'Stop!')
('Decoded sentence:', 'Halt!\n')
-
('Input sentence:', 'Stop!')
('Decoded sentence:', 'Halt!\n')
